# Simple graphical fitting

In [ ]:
import os
import logging

import bokeh
import holoviews as hv
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

import cellpy
from cellpy import cellreader
from cellpy.utils import ica

%matplotlib inline
hv.extension('bokeh')

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

print(f"pandas: {pd.__version__}")
print(f"cellpy: {cellpy.__version__}")
print(f"holoviews: {hv.__version__}")

my_data = cellreader.CellpyData()
filename = "../../../testdata/hdf5/20160805_test001_45_cc.h5"
assert os.path.isfile(filename)
my_data.load(filename)
my_data.set_mass(0.1)

## 1. Retrieve dQ/dV data

### What is the best way to retrieve dQ/dV data?

In [ ]:
comb_ica = ica.dqdv_frames(my_data, split=False, tidy=False)
comb_ica.head()

In [ ]:
cycle_number = 1
ax = comb_ica[cycle_number].plot(x="voltage")
ax.legend([f"cycle {cycle_number}"])

In [ ]:
comb_ica_charge, comb_ica_discharge = ica.dqdv_frames(my_data, split=True, tidy=False, )

In [ ]:
comb_ica_charge.head()

In [ ]:
cycle_number = 1
ax = comb_ica_charge[["voltage", cycle_number]].plot(x=("voltage", "v"))
ax.legend([f"charge cycle {cycle_number}"])

ax = comb_ica_discharge[["voltage", cycle_number]].plot(x=("voltage", "v"))
ax.legend([f"discharge cycle {cycle_number}"])

In [ ]:
tidy_ica_charge, tidy_ica_discharge = ica.dqdv_frames(my_data, split=True, tidy=True)

In [ ]:
cycle_number = 1
ax = tidy_ica_charge.loc[tidy_ica_charge.cycle == cycle_number, ["voltage", "dq"]].plot(x="voltage")
ax.legend([f"charge cycle {cycle_number}"])

### Selected retrieve method

For fitting, it is most convinient to have individual datasets for charge and discharge. It is also convinient to have the data in a form where we can easily plot all the data to get an overview. And it should be easy to extract single cycles.

In [ ]:
tidy_ica_charge, tidy_ica_discharge = ica.dqdv_frames(my_data, split=True, tidy=True)

cycle_number = 1
dq_charge_1 = tidy_ica_charge.loc[tidy_ica_charge.cycle == cycle_number, ["voltage", "dq"]]

### Observed improvements needed

- `dqdv_frames` gives a warning if step is not found - make that optional (warnings=False or True)
- inconsistent column names for `dqdv_frames` with `split=True` *vs* `split=False`
    - split=True creates a "interpolated" frame with one common x-column ("voltage", "v) and one column pr cycle ("1", "dq")
    - split=False creates x,y pair values with columns ("1", "voltage") and ("1", "dq")
- need to rename the function to something more intuitive
    - for example: `make_dqdv_frame` or `retrieve_dqdv_frame`

## 2. Interact and plot the data

### As simple as possible

In [ ]:
pd.options.plotting.backend = "matplotlib"
ax_mplib = dq_charge_1.plot(x="voltage", style="-o")
ax_mplib.legend([f"charge cycle {cycle_number}"])

In [ ]:
pd.options.plotting.backend = "hvplot"
ax_hvplot = dq_charge_1.plot(x="voltage") * dq_charge_1.plot(x="voltage", kind="scatter")
# Unfortunately, the df.plot() method returns a hv object with a copy of the dataframe (potential memory leak)
# We therefore chose to use the hv.Curve() etc. methods directly instead.
ax_hvplot

### Using holoviews directly
(since dataframe.plot() returns a copy of the data)

In [ ]:
cycle_label = "Cycle 1 (charge)"
value_dims = [
    hv.Dimension("dq", label="dQ", unit="mAh/g/V")
]  # the y-axis
key_dims = [
    hv.Dimension("voltage", label="Voltage", unit="V vs. Li/Li+")
]    # the x-axis
z_dims = key_dims + value_dims

v = hv.Scatter(data=dq_charge_1, kdims=key_dims, vdims=value_dims, label=cycle_label).opts(
    width=800,
    height=400,
    marker="o",
    size=8,
    tools = ['hover']
)
vv = (v * hv.Curve(v)).opts(title="ICA plot")
vv

### Interactive point picking

In [ ]:
import holoviews as hv
from holoviews import opts, streams
from holoviews.plotting.links import DataLink

In [ ]:
# A small test to check if it is possible to pick points easily.
# The short answer is ... yes, but no idea how to get the picked data out.

cycle_label = "Cycle 1 (charge)"
value_dims = [
    hv.Dimension("dq", label="dQ", unit="mAh/g/V")
]
key_dims = [
    hv.Dimension("voltage", label="Voltage", unit="V vs. Li/Li+"),
    hv.Dimension("dq", label="dQ", unit="mAh/g/V")
]

peaks = pd.DataFrame({"voltage": [0.4, 0.5], "dq": [1.7e5, 1.5e5]})
points = hv.Points(data=peaks, kdims=key_dims).opts(size=12, color="red")

v = hv.Scatter(data=dq_charge_1, kdims=key_dims, vdims=value_dims, label=cycle_label).opts(
    width=600,
    height=400,
    marker="o",
    size=8,
    tools = ['hover']
)
vv = (v * hv.Curve(v)).opts(title="ICA plot")


point_stream = streams.PointDraw(num_objects=2, source=points, empty_value='black')

table = hv.Table(points, key_dims)
DataLink(points, table)

(vv * points + table).opts(
    opts.Layout(merge_tools=False),
    opts.Points(active_tools=['point_draw'], size=10),
    opts.Table(editable=True))



In [ ]:
table.data

#### FAILED

## Use Bokeh directly?

In [ ]:
import numpy as np

from bokeh.plotting import figure, output_notebook, show

# prepare some data
N = 4000
x = np.random.random(size=N) * 100
y = np.random.random(size=N) * 100
radii = np.random.random(size=N) * 1.5
colors = [
    "#%02x%02x%02x" % (int(r), int(g), 150) for r, g in zip(50+2*x, 30+2*y)
]

# output to static HTML file (with CDN resources)
output_file("color_scatter.html", title="color_scatter.py example", mode="cdn")

TOOLS = "crosshair,pan,wheel_zoom,box_zoom,reset,box_select,lasso_select"

# create a new plot with the tools above, and explicit ranges
p = figure(tools=TOOLS, x_range=(0, 100), y_range=(0, 100))

# add a circle renderer with vectorized colors and sizes
p.circle(x, y, radius=radii, fill_color=colors, fill_alpha=0.6, line_color=None)

# show the results
show(p)